In [1]:
# Installing the Conda
!conda update -n base -c defaults conda --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-package-handling-1.3.11|           py37_0         260 KB
    certifi-2019.6.16          |           py37_1         156 KB
    libxml2-2.9.9              |       hea5a465_1         2.0 MB
    ca-certificates-2019.5.15  |                1         134 KB
    chardet-3.0.4              |        py37_1003         173 KB
    python-3.7.4               |       h265db76_1        36.5 MB
    pysocks-1.7.0              |           py37_0          29 KB
    python-libarchive-c-2.8    |          py37_13          23 KB
    conda-4.7.11               |           py37_0         3.0 MB
    ncurses-6.1                |       he6710b0_1         958 KB
    pycparser-2.19             |           py37_0         172 KB
    

In [2]:
%pip install beautifulsoup4 

     |████████████████████████████████| 102kB 18.4MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import random 
import requests
from bs4 import BeautifulSoup

In [4]:
%pip install geopy

     |████████████████████████████████| 102kB 17.5MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install geocoder

     |████████████████████████████████| 102kB 14.7MB/s ta 0:00:01
     |████████████████████████████████| 829kB 29.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [7]:
import geocoder

In [8]:
df = pd.read_csv('Toronto_Part-A.csv')

In [9]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Loading the Dataframe "toronto_base.csv" from Part-A

In [10]:
df_Toronto = pd.read_csv('Toronto_Part-A.csv')
df_Toronto.head(7)

,Unnamed: 0,Postcode,Borough,Neighbourhood
0,0,M9A,Etobicoke,Islington Avenue
1,1,M4H,East York,Thorncliffe Park
2,2,M1B,Scarborough,"Rouge, Malvern"
3,3,M9L,North York,Humber Summit
4,4,M4Y,Downtown Toronto,Church and Wellesley
5,5,M9N,York,Weston
6,6,M3J,North York,"Northwood Park, York University"


In [11]:
df_geospatial_data = pd.read_csv('http://cocl.us/Geospatial_data')
df_geospatial_data.columns = ['Postcode', 'Latitude', 'Longitude']
df_geospatial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_Post = pd.merge(df_Toronto, df_geospatial_data, on=['Postcode'], how='inner')
df_Toronto = df_Post[[ 'Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude',]].copy()
df_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,M4H,East York,Thorncliffe Park,43.705369,-79.349372
2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
3,M9L,North York,Humber Summit,43.756303,-79.565963
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160


### Creating a Map of Toronto

In [16]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# for the city Toronto, latitude and longtitude are manually extracted via google search
Toronto_latitude = 43.6532; Toronto_longitude = -79.3832
map_Toronto = folium.Map(location = [Toronto_latitude, Toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'],
                                           df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Toronto)  
    
map_Toronto

In [22]:
# @hiddel_cell
CLIENT_ID = 'YOHRI5JQ3KIYWOMLHROI3S4ZUVDG0JB2SBLQWVSP0SNKUOM0' #  Foursquare ID
CLIENT_SECRET = 'ZIL0PGRZQMV45FDOVM0UZAWHTNJ0A0WCPTPDXQIERGOR4ZQD' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [23]:
Scarborough_data = df_Toronto[df_Toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head(7)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302
2,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
3,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476
4,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
5,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
6,M1J,Scarborough,Scarborough Village,43.744734,-79.239476


### Map of Scarborough and Its Neighbourhoods

In [24]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [26]:
map_Scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Scarb)  
    
map_Scarb

### Get the top 100 venues in the neighborhood 'Steeles West', from Scarborough

In [27]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge, Malvern" are 43.806686299999996, -79.19435340000001.


In [28]:
# @hiddel_cell
CLIENT_ID = 'YOHRI5JQ3KIYWOMLHROI3S4ZUVDG0JB2SBLQWVSP0SNKUOM0' #  Foursquare ID
CLIENT_SECRET = 'ZIL0PGRZQMV45FDOVM0UZAWHTNJ0A0WCPTPDXQIERGOR4ZQD' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YOHRI5JQ3KIYWOMLHROI3S4ZUVDG0JB2SBLQWVSP0SNKUOM0
CLIENT_SECRET:ZIL0PGRZQMV45FDOVM0UZAWHTNJ0A0WCPTPDXQIERGOR4ZQD


In [29]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [31]:
results = requests.get(url).json()

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
5,DAVIDsTEA,Tea Room,43.776613,-79.258516
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,Hot Topic,Clothing Store,43.775450,-79.257929
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighborhood in Scarborough

In [36]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighbourhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge, Malvern
Tam O'Shanter
Upper Rouge
Cliffcrest, Cliffside
Highland Creek, Rouge Hill, Port Union
Maryvale, Wexford
Scarborough Village
Agincourt North, Milliken
Clairlea, Golden Mile, Oakridge
Woburn
Ionview, Kennedy Park
Birch Cliff
Morningside, West Hill
Agincourt
Dorset Park, Scarborough Town Centre, Wexford Heights


In [37]:
Scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Tam O'Shanter,43.781638,-79.304302,The Royal Chinese Restaurant 避風塘小炒,43.780505,-79.298844,Chinese Restaurant
2,Tam O'Shanter,43.781638,-79.304302,Remezzo Italian Bistro,43.778649,-79.308264,Italian Restaurant
3,Tam O'Shanter,43.781638,-79.304302,Eight Noodles,43.778234,-79.308299,Noodle House
4,Tam O'Shanter,43.781638,-79.304302,Kub Khao,43.780438,-79.299837,Thai Restaurant
5,Tam O'Shanter,43.781638,-79.304302,TD Canada Trust,43.779169,-79.303617,Bank
6,Tam O'Shanter,43.781638,-79.304302,Little Caesars,43.780563,-79.298624,Pizza Place
7,Tam O'Shanter,43.781638,-79.304302,KFC,43.779440,-79.303371,Fast Food Restaurant
8,Tam O'Shanter,43.781638,-79.304302,Popeyes Louisiana Kitchen,43.780476,-79.298460,Fried Chicken Joint
9,Tam O'Shanter,43.781638,-79.304302,Gusto Pizza,43.783607,-79.298983,Pizza Place


In [38]:
Scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
51,Agincourt,43.79420,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
52,Agincourt,43.79420,-79.262029,Twilight,43.791999,-79.258584,Lounge
53,Agincourt,43.79420,-79.262029,Mark's,43.791179,-79.259714,Clothing Store
54,Agincourt,43.79420,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink
55,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Kairali,43.754915,-79.276945,Indian Restaurant
56,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant
57,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Karaikudi Chettinad South Indian Restaurant,43.756042,-79.276276,Indian Restaurant
58,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Pho Vietnam,43.757770,-79.278572,Vietnamese Restaurant
59,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,Big Al's Pet Supercentre,43.759279,-79.278325,Pet Store
60,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,El Pulgarcito,43.754790,-79.277064,Latin American Restaurant


In [39]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",2,2,2,2,2,2
Birch Cliff,4,4,4,4,4,4
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Cliffcrest, Cliffside",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"Highland Creek, Rouge Hill, Port Union",2,2,2,2,2,2
"Ionview, Kennedy Park",4,4,4,4,4,4
"Maryvale, Wexford",4,4,4,4,4,4


In [41]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 43 uniques categories.


In [42]:
# one hot encoding
Scarb_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarb_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarb_onehot.columns[-1]] + list(Scarb_onehot.columns[:-1])
Scarb_onehot = Scarb_onehot[fixed_columns]

Scarb_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Tam O'Shanter,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Tam O'Shanter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [43]:
Scarb_onehot.shape

(61, 44)

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
Scarb_grouped = Scarb_onehot.groupby('Neighborhood').mean().reset_index()
Scarb_grouped.head(10)

,Neighborhood,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.0,0.250000,0.000000,0.000000,0.00,0.000000,0.25,0.00,0.00,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.25,0.00,0.000000,0.0,0.000000
1,"Agincourt North, Milliken",0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.500000,0.000000,0.0,0.000000,0.5,0.000000,0.00,0.00,0.000000,0.0,0.000000
2,Birch Cliff,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.00,0.00,0.25,0.0,0.00,0.000000,0.000000,0.0,0.25,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.25,0.00,0.000000,0.0,0.000000
3,"Clairlea, Golden Mile, Oakridge",0.0,0.111111,0.0,0.0,0.000000,0.222222,0.111111,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.111111,0.0,0.00,0.00,0.000000,0.111111,0.0,0.0,0.000000,0.00,0.000000,0.111111,0.000000,0.00,0.0,0.0,0.0,0.111111,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.00,0.111111,0.0,0.000000
4,"Cliffcrest, Cliffside",0.5,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.5,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.0,0.000000
5,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.166667,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.333333,0.000000,0.0,0.0,0.166667,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.0,0.166667
6,"Highland Creek, Rouge Hill, Port Union",0.0,0.000000,0.0,0.5,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.5,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.0,0.000000
7,"Ionview, Kennedy Park",0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.250000,0.00,0.25,0.00,0.0,0.25,0.000000,0.000000,0.0,0.00,0.25,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.0,0.000000
8,"Maryvale, Wexford",0.0,0.250000,0.0,0.0,0.250000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.25,0.000000,0.0,0.000000
9,"Morningside, West Hill",0.0,0.000000,0.0,0.0,0.142857,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.0,0.00,0.142857,0.000000,0.0,0.00,0.00,0.000000,0.142857,0.0,0.0,0.000000,0.00,0.142857,0.000000,0.142857,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.142857,0.0,0.142857,0.00,0.00,0.000000,0.0,0.000000


In [52]:
Num_Top_Venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(Num_Top_Venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarb_grouped['Neighborhood']

for ind in np.arange(Scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
1,"Agincourt North, Milliken",Playground,Park,Vietnamese Restaurant,Clothing Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,"Clairlea, Golden Mile, Oakridge",Bus Line,Intersection,Soccer Field,Bakery,Fast Food Restaurant,Bus Station,Park,Metro Station,College Stadium,General Entertainment
4,"Cliffcrest, Cliffside",American Restaurant,Motel,Intersection,Hobby Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
5,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,"Highland Creek, Rouge Hill, Port Union",Moving Target,Bar,Vietnamese Restaurant,Hobby Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
7,"Ionview, Kennedy Park",Coffee Shop,Hobby Shop,Department Store,Chinese Restaurant,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Convenience Store
8,"Maryvale, Wexford",Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,"Morningside, West Hill",Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Fried Chicken Joint


### Run k-means to cluster the neighborhoods into 5 clusters

In [57]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

Scarb_data = Scarborough_data.drop(13)
# set number of clusters
kclusters = 5

Scarb_grouped_clustering = Scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([1, 3, 1, 1, 1, 1, 4, 0, 1, 1], dtype=int32)

In [59]:
Scarb_merged = Scarb_data

# add clustering labels
Scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarb_merged = Scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Scarb_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hobby Shop,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store,Convenience Store,College Stadium
1,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302,3,Pizza Place,Italian Restaurant,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Noodle House,Bank,Bus Line
2,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476,1,American Restaurant,Motel,Intersection,Hobby Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
4,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,Moving Target,Bar,Vietnamese Restaurant,Hobby Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
5,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,1,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Hobby Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
7,M1V,Scarborough,"Agincourt North, Milliken",43.815252,-79.284577,0,Playground,Park,Vietnamese Restaurant,Clothing Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,1,Bus Line,Intersection,Soccer Field,Bakery,Fast Food Restaurant,Bus Station,Park,Metro Station,College Stadium,General Entertainment
9,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store


### Visualize the clusters in the map

In [67]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Scarb_merged['Latitude'], Scarb_merged['Longitude'], 
                                  Scarb_merged['Neighbourhood'], Scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each of the five clusters

In [62]:
Scarb_merged.loc[Scarb_merged['Cluster Labels'] == 0, 
                 Scarb_merged.columns[[1] + list(range(5, Scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,0,Playground,Park,Vietnamese Restaurant,Clothing Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
14,Scarborough,0,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [63]:
Scarb_merged.loc[Scarb_merged['Cluster Labels'] == 1, 
                 Scarb_merged.columns[[1] + list(range(5, Scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hobby Shop,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store,Convenience Store,College Stadium
2,Scarborough,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Scarborough,1,American Restaurant,Motel,Intersection,Hobby Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
4,Scarborough,1,Moving Target,Bar,Vietnamese Restaurant,Hobby Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
5,Scarborough,1,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
8,Scarborough,1,Bus Line,Intersection,Soccer Field,Bakery,Fast Food Restaurant,Bus Station,Park,Metro Station,College Stadium,General Entertainment
9,Scarborough,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
12,Scarborough,1,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,Fried Chicken Joint


In [64]:
Scarb_merged.loc[Scarb_merged['Cluster Labels'] == 2, 
                 Scarb_merged.columns[[1] + list(range(5, Scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,2,Coffee Shop,Hobby Shop,Department Store,Chinese Restaurant,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Convenience Store


In [65]:
Scarb_merged.loc[Scarb_merged['Cluster Labels'] == 3, 
                 Scarb_merged.columns[[1] + list(range(5, Scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Pizza Place,Italian Restaurant,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Noodle House,Bank,Bus Line
11,Scarborough,3,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [66]:
Scarb_merged.loc[Scarb_merged['Cluster Labels'] == 4, 
                 Scarb_merged.columns[[1] + list(range(5, Scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Hobby Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
